In [243]:
import pandas as pd
import numpy as np
import json

In [244]:
# Rutas de los datasets originales
ruta_credits = r'C:\HENRY\PROYECTOS HENRY 11\Movies\credits1.json'
ruta_credits_csv = r'C:\HENRY\PROYECTOS HENRY 11\Movies\movies_dataset.csv'
ruta_movies = r'C:\HENRY\PROYECTOS HENRY 11\Movies\movies2.json'

In [245]:
credits_lista = []
with open(ruta_credits, 'rt', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if line:
            try:
                credits_lista.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error en la línea {line_num}: {line}")
                print(f"Detalle del error: {e}")
df_credits = pd.DataFrame(credits_lista)


In [246]:
df_credits.head()

,Cast,Crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ''Savannah 'Vanna...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [247]:
#df = pd.read_csv(ruta_credits_csv)
#df.to_json('movies2.json', orient='records', lines=True)

In [248]:
movies_lista = []
with open(ruta_movies, 'rt', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if line:
            try:
                movies_lista.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error en la línea {line_num}: {line}")
                print(f"Detalle del error: {e}")
df_movies = pd.DataFrame(movies_lista)


In [249]:
df_movies.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,False,7.7,5415
1,False,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [250]:
df_movies['belongs_to_collection'].loc[2]

{'id': 119050,
 'name': 'Grumpy Old Men Collection',
 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg',
 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}

In [251]:
df_credits.columns

Index(['Cast', 'Crew', 'id'], dtype='object')

In [252]:
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [253]:
#Cambiar valores nulos en revenue por 0
df_movies['revenue'] = np.where(pd.isnull(df_movies['revenue']), 0, df_movies['revenue'])

In [254]:
#Cambiar valores nulos en revenue por 0
df_movies['budget'] = np.where(pd.isnull(df_movies['budget']), 0, df_movies['budget'])

In [255]:
df_movies['release_date'].dropna()

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45457    1991-05-13
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45379, dtype: object

DESEMPAQUETAR

In [256]:
# Función para limpiar y cargar las cadenas JSON
def clean_and_load_json(s):
    if isinstance(s, str):
        try:
            return json.loads(s.replace("'", "\""))
        except json.JSONDecodeError:
            return None
    return s

# Aplicar la función a la columna
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(clean_and_load_json)

# Ahora intentamos desanidar
def desanidar_columna(df, columna, prefijo):
    desanidado = pd.json_normalize(df[columna])
    desanidado.columns = [f"{prefijo}_{col}" for col in desanidado.columns]
    df = df.drop(columns=[columna]).join(desanidado)
    return df

In [257]:
# Desanidar y renombrar las columnas
df_movies = desanidar_columna(df_movies, 'belongs_to_collection', 'btc')

In [258]:
#df_movies = desanidar_columna(df_movies, 'production_companies', 'pc')


In [259]:
df_movies.columns

Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count', 'btc_id',
       'btc_name', 'btc_poster_path', 'btc_backdrop_path'],
      dtype='object')

In [260]:
df_movies.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,status,tagline,title,video,vote_average,vote_count,btc_id,btc_name,btc_poster_path,btc_backdrop_path
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,Released,None,Toy Story,False,7.7,5415,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413,NaN,NaN,NaN,NaN


In [261]:
# Cambiamos el tipo de dato de la columna 'release_date' str a datetime
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

# Verificamos el tipo de dato de la columna
print(type(df_movies['release_date'].loc[2]))


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [262]:
#Creamos la columna 'release_year', la cual se obtiene de sacar el año de la columna 'release_date'
df_movies['release_year']=df_movies['release_date'].dt.year

In [263]:
#Creamos columna 'return', la cual es el resultado de revenue / budget
df_movies['return']=df_movies['revenue']/df_movies['budget']

In [264]:
#Prueba
print(df_movies['revenue'].loc[1])
print(df_movies['budget'].loc[1])
print(df_movies['return'].loc[1])

262797249
65000000
4.0430346


In [265]:
#Eliminamos algunas columnas
df_movies = df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [266]:
df_movies.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'btc_id', 'btc_name', 'btc_poster_path',
       'btc_backdrop_path', 'release_year', 'return'],
      dtype='object')

In [267]:
#Transformando DataFrame a parquet 

#df_credits.to_parquet('credits.parquet', engine='pyarrow')


In [268]:
#df_movies.to_json('movies.json')

In [269]:
df_movies.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'btc_id', 'btc_name', 'btc_poster_path',
       'btc_backdrop_path', 'release_year', 'return'],
      dtype='object')

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
        Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [270]:
df_movies['original_language'].unique()

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo',
       'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo',
       'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv',
       'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms',
       'sq', None, 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay',
       'kn', 'eu', 'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu',
       'lb', 'si'], dtype=object)

In [271]:

def cantidad_filmaciones_mes1( Mes ):
    meses = {
        1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo",
        6: "junio", 7: "julio", 8: "agosto", 9: "septiembre", 
        10: "octubre", 11: "noviembre", 12: "diciembre"
    }
    peliculas_espanol = df_movies[df_movies['original_language'] == 'es']
    peliculas_mes = peliculas_espanol[peliculas_espanol['release_date'].dt.month == Mes]
    print(f"{len(peliculas_mes)} películas fueron estrenadas en el mes de {meses[Mes]}")



In [272]:
cantidad_filmaciones_mes1(4)

97 películas fueron estrenadas en el mes de abril


In [273]:
#Función que calcula la cantidad de peliculas estrenadas en un mes determinado
def cantidad_filmaciones_mes( Mes ):
    Mes= Mes.lower() #Convierte el valor ingresado a minúsculas
    #asignamos un valor a cada mes en español que entre como parámetro
    meses = {
        "enero": 1, "febrero": 2, "marzo": 3, "abril":4, "mayo": 5,
        "junio":6, "julio":7, "agosto":8, "septiembre":9, 
         "octubre":10, "noviembre":11, "diciembre":12
    }
    if(Mes not in meses):#Si el mes seleccionado no está entre las opciones
        print("Ingrese un mes en español")
        return
    peliculas_mes = (df_movies[df_movies['release_date'].dt.month==meses[Mes]])
    print(f"{len(peliculas_mes)} películas fueron estrenadas en el mes de {Mes}")

In [274]:
cantidad_filmaciones_mes('ENERO')

5913 películas fueron estrenadas en el mes de enero


In [275]:
print(len(df_movies[df_movies['release_date'].dt.month==1]))

5913


def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
            Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [276]:
#Función que calcula ca cantidad de peliculas estrenadas en un día determinado
def cantidad_filmaciones_dia( Dia ):
    Dia= Dia.lower() #Convierte el valor ingresado a minúsculas#Convierte el valor ingresado a minúsculas
    #asignamos un valor a cada día en español que entre como parámetro
    Dias = {
        "lunes": "Monday", "martes": "Tuesday", "miercoles": "Wednesday", "jueves":"Thursday", "viernes": "Friday",
        "sabado":"Saturday", "domingo":"Sunday"
    }
    if(Dia not in Dias):#Si el día seleccionado no está entre las opciones
        print("Ingrese un día en español")
        return
    peliculas_dia = (df_movies[df_movies['release_date'].dt.day_name()==Dias[Dia]])
    print(f"{len(peliculas_dia)} películas fueron estrenadas en el mes de {Dia}")

In [277]:
#Probando la función
cantidad_filmaciones_dia("lunes")

3503 películas fueron estrenadas en el mes de lunes


In [240]:
#Prueba
len(df_movies[df_movies['release_date'].dt.day_name()=='Monday'])

3503

In [280]:
type(df_movies['release_date'].loc[2])

pandas._libs.tslibs.timestamps.Timestamp

: 